In [1]:
from brainbox.task import trials
from one.api import ONE
import pandas as pd
import numpy as np
import GEMA

one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)
eid = '58b1e920-cfc8-467e-b28b-7654a55d0977'

In [24]:
def get_raster_and_psth(eid):
    #Load trials into df
    dat = one.load_dataset(eid, 'alf/_ibl_trials.table.pqt')
    df=pd.DataFrame(dat)
    #Choose one type of event
    trials_=df.loc[(df['choice'] == 1) & (df['contrastLeft'] == 0.0)].index
    #df=df.loc[(df['choice'] == 1) & (df['contrastLeft'] == 1.0)]
    events=df["stimOn_times"]
    #Get spike_times
    spike_times=one.load_dataset(eid, 'alf/probe00/pykilosort/spikes.times.npy')
    #Get clusters
    spike_clusters=one.load_dataset(eid, 'alf/probe00/pykilosort/spikes.clusters.npy')
    print(len(spike_clusters))
    print(spike_times.shape)
    #Get binned rasters
    raster_dict={}
    time_vec={}
    for n in list(np.unique(spike_clusters)):
        neuron=np.where(spike_clusters==n)
        if len(list(neuron[0]))!=0:
            #print(len(list(neuron[0])))
            raster=trials.get_event_aligned_raster(spike_times[neuron], events, tbin=0.1, values=None, epoch=[-0.5, 2], bin=True)
            #print(raster)
            if np.isnan(raster[0]).any()==False:
                raster_dict[n]=raster[0]
        time_vec=raster[1]
    psth_dict={}
    for k in raster_dict.keys():
        psth=trials.get_psth(raster_dict[k],trial_ids=trials_)
        psth_dict[k]=psth
    return raster_dict,psth_dict,time_vec

raster_dict, psth_dict,time_vec=get_raster_and_psth(eid)

from scipy.stats import zscore
psth_mat=[]
for k in psth_dict.keys():
    psth_mat.append(psth_dict[k][0])
psth_mat=np.array(psth_mat)

np.save('psth_arr_z.npy',psth_mat)

5469038
(5469038,)


In [2]:
import GEMA

psth_mat=np.load('psth_arr_z.npy')
import time

start=time.time()
print(start)

from sklearn import preprocessing
map_size = 10
period = 10000 
learning_rate = 0.1
print(np.isnan(psth_mat))
normalize_data = 'none'
datos = preprocessing.normalize(psth_mat)

main_map = GEMA.Map(datos,
                    map_size,
                    period, 
                    initial_lr=learning_rate, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    normalization = normalize_data,
                    presentation= 'random',
                    weights = 'random')

end=time.time()
print(end-start)

#Color SOM
from sklearn import preprocessing
datos_ = np.random.randint(0, 255, (100, 3))
datos_ = preprocessing.normalize(datos_)
main_map_colors = GEMA.Map(datos_, 
                    map_size, 
                    period, 
                    initial_lr=learning_rate, 
                    initial_neighbourhood = 0,
                    distance = 'euclidean',
                    use_decay= False,
                    normalization = normalize_data,
                    presentation= 'random',
                    weights = 'random')
from matplotlib.pyplot import imshow
image_matrix = (main_map_colors.weights-main_map_colors.weights.min())/(main_map_colors.weights.max()-main_map_colors.weights.min())

plt_arr=[]
for j in range(0,10):
    plt_lst=[]
    for i in range(0,10):
        plt_lst.append(main_map.weights[i,j,:])
    plt_arr.append(plt_lst)
plt_arr=np.array(plt_arr)
print(plt_arr.shape)
np.save('psth_prototypes_arr_z.npy',plt_arr)

import pickle
bmu_dct={}
for i in range(0,10):
    for j in range(0,10):
        bmu_dct[(j,i)]=[]
for k in range(0,datos.shape[0]):
    grid_index=main_map.calculate_bmu(datos[k,:])[1]
    bmu_dct[grid_index].append(k)
with open('bmu_dct_z.pickle', 'wb') as handle:
    pickle.dump(bmu_dct, handle, protocol=pickle.HIGHEST_PROTOCOL)
print(bmu_dct)

alpha_im=np.zeros((10,10,4))
for i in range(0,10):
    for j in range(0,10):
        alpha_im[i,j,:3]=image_matrix[i,j,:]
        if len(bmu_dct[(i,j)])==0:
            alpha_im[i,j,3]=0.5
        else:
            alpha_im[i,j,3]=1
            
np.save('alpha_im_z.npy',alpha_im)

1678808883.3093705
[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]
TRAINING...


  0%|          | 0/10000 [00:00<?, ?it/s]

FINISHED.
2.082796573638916
TRAINING...


  0%|          | 0/10000 [00:00<?, ?it/s]

FINISHED.
(10, 10, 26)
{(0, 0): [149, 164, 171, 172, 232], (1, 0): [63, 89], (2, 0): [140, 191], (3, 0): [19, 47, 105], (4, 0): [155, 204], (5, 0): [35, 185, 212], (6, 0): [40, 66, 224, 226], (7, 0): [87, 88, 95, 146, 210, 227, 233, 239, 250], (8, 0): [230], (9, 0): [90, 98, 241, 247, 249], (0, 1): [136, 148], (1, 1): [], (2, 1): [67, 108, 114, 183], (3, 1): [107, 109], (4, 1): [], (5, 1): [], (6, 1): [223], (7, 1): [78], (8, 1): [], (9, 1): [221], (0, 2): [138, 150, 154, 202], (1, 2): [139, 219], (2, 2): [58, 106], (3, 2): [], (4, 2): [178, 193, 251], (5, 2): [37], (6, 2): [22, 252], (7, 2): [211], (8, 2): [157], (9, 2): [29, 159, 225], (0, 3): [15, 62, 97, 158], (1, 3): [74, 137, 184], (2, 3): [4, 43], (3, 3): [182], (4, 3): [32, 135], (5, 3): [245], (6, 3): [248], (7, 3): [123, 173], (8, 3): [168], (9, 3): [113, 231], (0, 4): [3, 23, 68, 82, 186, 201], (1, 4): [5, 52, 72], (2, 4): [117, 143, 144, 229], (3, 4): [33, 213], (4, 4): [116, 118], (5, 4): [122, 206], (6, 4): [42, 115], (7,

In [3]:
print(image_matrix.shape)

(10, 10, 3)


In [4]:
print(bmu_dct)

{(0, 0): [149, 164, 171, 172, 232], (1, 0): [63, 89], (2, 0): [140, 191], (3, 0): [19, 47, 105], (4, 0): [155, 204], (5, 0): [35, 185, 212], (6, 0): [40, 66, 224, 226], (7, 0): [87, 88, 95, 146, 210, 227, 233, 239, 250], (8, 0): [230], (9, 0): [90, 98, 241, 247, 249], (0, 1): [136, 148], (1, 1): [], (2, 1): [67, 108, 114, 183], (3, 1): [107, 109], (4, 1): [], (5, 1): [], (6, 1): [223], (7, 1): [78], (8, 1): [], (9, 1): [221], (0, 2): [138, 150, 154, 202], (1, 2): [139, 219], (2, 2): [58, 106], (3, 2): [], (4, 2): [178, 193, 251], (5, 2): [37], (6, 2): [22, 252], (7, 2): [211], (8, 2): [157], (9, 2): [29, 159, 225], (0, 3): [15, 62, 97, 158], (1, 3): [74, 137, 184], (2, 3): [4, 43], (3, 3): [182], (4, 3): [32, 135], (5, 3): [245], (6, 3): [248], (7, 3): [123, 173], (8, 3): [168], (9, 3): [113, 231], (0, 4): [3, 23, 68, 82, 186, 201], (1, 4): [5, 52, 72], (2, 4): [117, 143, 144, 229], (3, 4): [33, 213], (4, 4): [116, 118], (5, 4): [122, 206], (6, 4): [42, 115], (7, 4): [55, 112], (8, 4):

In [17]:
print(alpha_im.shape)

(10, 10, 4)
